# 섹션 3: Naive RAG 이해 및 구현

## 실습 목표
- RAG 파이프라인 전체 구현
- 문서 기반 Q&A 시스템 만들기
- LLM의 환각 문제를 RAG로 해결

## 사용 모델
- **Google Gemini 2.5 Flash Lite** (무료, Rate Limit 회피용 권장)

---
## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install langchain langchain-google-genai langchain-community chromadb -q

In [ ]:
# 라이브러리 임포트
import os
from getpass import getpass

# LangChain 관련
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document

print("라이브러리 로드 완료!")

In [ ]:
# API Key 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")

if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = getpass("Google API Key를 입력하세요: ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("API Key 설정 완료!")
print("API Key 발급: https://aistudio.google.com/app/apikey")

---
## 2. 샘플 문서 준비

실습용 학교 규칙 문서를 만들어봅시다.

In [ ]:
# 샘플 학교 규칙 문서
school_rules = """
# 서울AI고등학교 학칙

## 제1장 등교 및 출석

제1조 (등교시간)
1. 본교의 정규 등교 시간은 오전 8시 20분이다.
2. 8시 30분에 조회를 시작하므로 8시 25분까지는 교실에 착석해야 한다.
3. 8시 30분 이후 등교 시 지각으로 처리한다.

제2조 (조퇴 및 외출)
1. 조퇴 및 외출 시에는 반드시 담임선생님의 허가를 받아야 한다.
2. 조퇴/외출 신청서를 작성하여 교무실에 제출한다.
3. 보호자 동의가 필요한 경우 학교에서 연락을 취한다.

제3조 (결석)
1. 결석 시에는 당일 오전 9시까지 학교에 연락해야 한다.
2. 결석 사유서는 등교 후 3일 이내에 제출한다.
3. 무단결석 3회 시 학부모 상담을 실시한다.

## 제2장 복장 규정

제4조 (교복)
1. 등교 시에는 반드시 정규 교복을 착용한다.
2. 하복 기간: 5월 1일 ~ 9월 30일
3. 동복 기간: 10월 1일 ~ 4월 30일
4. 체육복은 체육 수업 시간에만 착용한다.

제5조 (두발 규정)
1. 두발은 단정하게 유지한다.
2. 염색 및 파마는 금지한다.
3. 머리 장식품은 화려하지 않은 것만 허용한다.

## 제3장 급식 및 점심시간

제6조 (급식)
1. 점심 급식 시간은 12시 30분부터 13시 20분까지이다.
2. 급식실에서는 조용히 식사한다.
3. 배식 순서는 학년별로 정해진 순서를 따른다.
   - 1학년: 12:30 ~ 12:50
   - 2학년: 12:50 ~ 13:10
   - 3학년: 13:10 ~ 13:30

제7조 (점심시간 활동)
1. 점심시간에는 도서관, 운동장, 동아리실을 이용할 수 있다.
2. 교실 밖 활동 시 학생증을 휴대한다.

## 제4장 휴대폰 사용

제8조 (휴대폰 관리)
1. 등교 시 담임선생님에게 휴대폰을 제출한다.
2. 휴대폰은 하교 시에 반환받는다.
3. 수업 중 휴대폰 사용 적발 시 1주일간 압수한다.

## 제5장 시험 및 평가

제9조 (정기고사)
1. 중간고사: 4월, 10월
2. 기말고사: 7월, 12월
3. 시험 중 부정행위 적발 시 해당 과목 0점 처리한다.

제10조 (수행평가)
1. 수행평가는 각 교과 담당 선생님의 계획에 따른다.
2. 수행평가 미제출 시 0점 처리한다.
3. 결시 시에는 추후 별도 평가를 실시한다.

## 제6장 방과후 활동

제11조 (방과후 학교)
1. 방과후 수업은 15시 30분부터 시작한다.
2. 신청은 매 학기 초에 온라인으로 진행한다.
3. 취소는 수업 시작 1주일 전까지 가능하다.

제12조 (동아리 활동)
1. 정규 동아리 활동은 매주 금요일 6-7교시에 진행한다.
2. 동아리 개설 조건: 최소 10명 이상의 학생, 지도교사 1명
3. 동아리 가입은 1학년 3월에 진행한다.

## 제7장 상벌점 제도

제13조 (상점)
1. 봉사활동 1시간당 상점 1점 부여
2. 선행 표창 시 상점 5점 부여
3. 교내 대회 입상 시 상점 3~10점 부여

제14조 (벌점)
1. 지각 1회: 벌점 1점
2. 무단결석 1회: 벌점 3점
3. 수업 방해 1회: 벌점 2점
4. 벌점 10점 누적 시 학부모 상담
5. 벌점 20점 누적 시 교내 봉사

## 부칙

1. 이 학칙은 2024년 3월 1일부터 시행한다.
2. 학칙에 명시되지 않은 사항은 학교 운영위원회에서 결정한다.
"""

print("학교 규칙 문서 준비 완료!")
print(f"문서 길이: {len(school_rules)}자")

---
## 3. RAG 파이프라인 구현

### Step 1: 문서 로드

In [ ]:
# Document 객체로 변환
documents = [Document(page_content=school_rules, metadata={"source": "school_rules"})]

print(f"로드된 문서 수: {len(documents)}")
print(f"문서 내용 미리보기: {documents[0].page_content[:200]}...")

### Step 2: 문서 분할 (Chunking)

In [ ]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,       # 각 청크의 최대 문자 수
    chunk_overlap=50,     # 청크 간 겹치는 문자 수
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 문서 분할
chunks = text_splitter.split_documents(documents)

print(f"생성된 청크 수: {len(chunks)}")
print("\n--- 청크 예시 ---")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n[청크 {i+1}]")
    print(chunk.page_content)
    print("-" * 50)

### Step 3: 임베딩 및 Vector DB 저장

In [ ]:
# Google 임베딩 모델 설정 (무료)
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
)

# 임베딩 테스트
test_text = "등교 시간이 몇 시인가요?"
test_vector = embeddings.embed_query(test_text)

print(f"임베딩 벡터 차원: {len(test_vector)}")
print(f"벡터 일부: {test_vector[:5]}")

In [ ]:
# Chroma Vector DB에 저장
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_school_rules"  # 로컬에 저장
)

print(f"Vector DB 생성 완료!")
print(f"저장된 문서 수: {vectorstore._collection.count()}")

### Step 4: 유사도 검색 테스트

In [ ]:
# 질문으로 유사 문서 검색
query = "등교 시간이 몇 시인가요?"

# 상위 3개 유사 문서 검색
relevant_docs = vectorstore.similarity_search(query, k=3)

print(f"질문: {query}")
print("\n--- 검색된 관련 문서 ---")
for i, doc in enumerate(relevant_docs):
    print(f"\n[문서 {i+1}]")
    print(doc.page_content)
    print("-" * 50)

In [ ]:
# 유사도 점수와 함께 검색
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

print(f"질문: {query}")
print("\n--- 유사도 점수 ---")
for doc, score in results_with_scores:
    print(f"\n점수: {score:.4f} (낮을수록 유사)")
    print(f"내용: {doc.page_content[:100]}...")

### Step 5: RAG 체인 구성

In [ ]:
# Gemini LLM 설정 (무료)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # Rate Limit 회피용 모델
    temperature=0,  # 일관된 답변을 위해 0으로 설정
    google_api_key=GOOGLE_API_KEY
)

# Retriever 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 상위 3개 문서 검색
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 모든 문서를 한번에 프롬프트에 포함
    retriever=retriever,
    return_source_documents=True  # 참조 문서도 반환
)

print("RAG 체인 설정 완료! (Gemini 2.5 Flash Lite 사용)")

### Step 6: 질문-답변 테스트

In [ ]:
# 질문 함수
def ask_question(question):
    result = qa_chain.invoke({"query": question})
    
    print(f"Q: {question}")
    print(f"\nA: {result['result']}")
    print("\n--- 참조 문서 ---")
    for i, doc in enumerate(result['source_documents'][:2]):
        print(f"[{i+1}] {doc.page_content[:100]}...")
    print("="*60)

In [ ]:
# 여러 질문 테스트
questions = [
    "등교 시간이 몇 시인가요?",
    "점심 급식 시간은 언제인가요?",
    "휴대폰 사용 규정은 어떻게 되나요?",
    "동아리 활동은 언제 하나요?",
    "벌점은 어떤 경우에 받나요?"
]

for q in questions:
    ask_question(q)
    print()

---
## 4. RAG vs 일반 LLM 비교

In [ ]:
# 일반 LLM에게 같은 질문 (Gemini 사용)
import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash-lite')  # Rate Limit 회피용 모델

def ask_without_rag(question):
    response = model.generate_content(question)
    return response.text

In [ ]:
# 비교 테스트
test_question = "서울AI고등학교의 점심 급식 시간은 언제인가요?"

print("=" * 60)
print("[일반 LLM의 답변]")
print(ask_without_rag(test_question))

print("\n" + "=" * 60)
print("[RAG를 사용한 답변]")
result = qa_chain.invoke({"query": test_question})
print(result['result'])

---
## 5. Streamlit으로 RAG 챗봇 만들기

In [ ]:
%%writefile rag_chatbot.py
"""
RAG 기반 학교 규칙 Q&A 챗봇 (Gemini 버전)
실행: streamlit run rag_chatbot.py
"""
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os

st.set_page_config(page_title="학교 규칙 Q&A", page_icon="📚")

st.title("학교 규칙 Q&A 챗봇 📚")
st.caption("학교 규칙에 대해 무엇이든 물어보세요! (Gemini 무료)")

# 샘플 학교 규칙
SCHOOL_RULES = '''
# 서울AI고등학교 학칙

제1조 (등교시간)
1. 본교의 정규 등교 시간은 오전 8시 20분이다.
2. 8시 30분에 조회를 시작하므로 8시 25분까지는 교실에 착석해야 한다.

제6조 (급식)
1. 점심 급식 시간은 12시 30분부터 13시 20분까지이다.
2. 1학년: 12:30~12:50, 2학년: 12:50~13:10, 3학년: 13:10~13:30

제8조 (휴대폰 관리)
1. 등교 시 담임선생님에게 휴대폰을 제출한다.
2. 휴대폰은 하교 시에 반환받는다.

제12조 (동아리 활동)
1. 정규 동아리 활동은 매주 금요일 6-7교시에 진행한다.

제14조 (벌점)
1. 지각 1회: 벌점 1점
2. 무단결석 1회: 벌점 3점
3. 벌점 10점 누적 시 학부모 상담
'''

# 사이드바
with st.sidebar:
    api_key = st.text_input("Google API Key", type="password",
                           help="https://aistudio.google.com/app/apikey")
    
    if st.button("대화 초기화"):
        st.session_state.messages = []
        st.rerun()
    
    st.divider()
    st.markdown("### 질문 예시")
    st.markdown("- 등교 시간이 몇 시인가요?")
    st.markdown("- 점심 급식은 언제인가요?")
    st.markdown("- 휴대폰 규정이 어떻게 되나요?")

# RAG 초기화 함수
@st.cache_resource
def init_rag(_api_key):
    os.environ["GOOGLE_API_KEY"] = _api_key
    
    # 문서 준비
    documents = [Document(page_content=SCHOOL_RULES)]
    
    # 청크 분할
    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
    chunks = splitter.split_documents(documents)
    
    # Vector DB 생성 (Google 임베딩)
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=_api_key
    )
    vectorstore = Chroma.from_documents(chunks, embeddings)
    
    # QA Chain 생성 (Gemini)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0,
        google_api_key=_api_key
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True
    )
    
    return qa_chain

# 메시지 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []

# 이전 메시지 표시
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# 새 메시지 처리
if prompt := st.chat_input("학교 규칙에 대해 질문하세요..."):
    if not api_key:
        st.error("API Key를 입력해주세요!")
        st.stop()
    
    # 사용자 메시지 표시
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # RAG 응답 생성
    with st.chat_message("assistant"):
        with st.spinner("검색 중..."):
            try:
                qa_chain = init_rag(api_key)
                result = qa_chain.invoke({"query": prompt})
                
                answer = result['result']
                st.markdown(answer)
                
                # 참조 문서 표시
                with st.expander("참조 문서 보기"):
                    for i, doc in enumerate(result['source_documents']):
                        st.markdown(f"**[문서 {i+1}]**")
                        st.markdown(doc.page_content)
                        st.divider()
                
                st.session_state.messages.append({"role": "assistant", "content": answer})
                
            except Exception as e:
                st.error(f"오류: {str(e)}")

---
## 6. 실습 과제

### 과제 1: 나만의 문서로 RAG 만들기

아래에 본인만의 문서를 작성하고 RAG 시스템을 만들어보세요.

In [ ]:
# 나만의 문서 작성 (예: 게임 공략, 요리 레시피, 취미 정보 등)
my_document = """
# 여기에 본인만의 문서를 작성하세요

예시:
# 마인크래프트 초보자 가이드

## 첫날 생존
1. 나무를 먼저 채집합니다.
2. 작업대를 만듭니다.
...
"""

# TODO: 위 문서로 RAG 시스템 구현
# 1. Document 객체 생성
# 2. 청크 분할
# 3. Vector DB 저장
# 4. QA Chain 생성
# 5. 질문-답변 테스트

### 과제 2: 파일에서 문서 로드하기

텍스트 파일을 읽어서 RAG 시스템에 적용해보세요.

In [ ]:
# 파일 로드 예시
from langchain_community.document_loaders import TextLoader

# loader = TextLoader("my_document.txt", encoding="utf-8")
# documents = loader.load()

# TODO: 파일을 로드하고 RAG 시스템 구현

---
## 7. 모둠 활동

### "RAG가 필요한 주제 선정하기"

조별로 다음을 논의하고 작성해보세요:

1. **주제**: 어떤 RAG 시스템을 만들고 싶은가?
2. **문서**: 어떤 문서가 필요한가?
3. **질문 예시**: 사용자가 할 수 있는 질문은?
4. **기대 효과**: 이 시스템의 장점은?

In [ ]:
# 모둠 활동 결과 작성
group_project = {
    "팀명": "",
    "주제": "",
    "필요 문서": [],
    "질문 예시": [],
    "기대 효과": ""
}

# 예시
example_project = {
    "팀명": "맛있는 AI",
    "주제": "한식 요리 레시피 Q&A 봇",
    "필요 문서": ["김치찌개 레시피", "된장찌개 레시피", "불고기 레시피"],
    "질문 예시": ["김치찌개에 뭘 넣어야 해?", "불고기 양념 비율이 어떻게 돼?"],
    "기대 효과": "요리 초보자도 쉽게 한식 요리 가능"
}

print("예시 프로젝트:")
for k, v in example_project.items():
    print(f"  {k}: {v}")

---
## 정리

### 오늘 배운 내용

1. **RAG 개념**: 검색 + 생성으로 LLM 한계 극복
2. **파이프라인**: Load → Split → Embed → Store → Retrieve → Generate
3. **임베딩**: 텍스트를 벡터로 변환 (Google Embedding)
4. **Vector DB**: Chroma로 문서 저장 및 검색
5. **LangChain**: RAG 체인 구성

### 사용 API
- **Google Gemini 2.5 Flash Lite** (무료, Rate Limit 회피용 권장)
- **Google Embedding** (무료)
- API Key: https://aistudio.google.com/app/apikey
